### Welcome to the ProtoSyn.jl examples

# 11 - Distributed Computing

By being developed in Julia, ProtoSyn enjoys some of the features naturally provided by the language, such as easy SIMD and GPU acceleration, a rich package environment and, among others, access to high level parallel and distributed computing routines. In this example we will take a look on how to launch and gather several decoys of the same Monte Carlo simulation. By having multiple decoys in parallel, given the random nature of the algorithm, we can have a greater confidence in the complete sampling of the conformational space, and that the obtained result is real. In order to do this, we first need to load the Distributed package and define the number of CPU workers (this number should be lower than the number of cores available).

In [ ]:
using Distributed

In [ ]:
n_workers = 4;
addprocs(n_workers);

The set-up of the code now needs to be replicated in each worker, using the `@everywhere` macro.

In [ ]:
@everywhere using ProtoSyn;

In [ ]:
@everywhere begin
    pose                 = ProtoSyn.Peptides.load("data/2a3d.pdb")
    selection            = an"C" | an"N"
    probability_mutation = 1/count(selection(pose))
    dihedral_mutator     = ProtoSyn.Mutators.DihedralMutator(randn, probability_mutation, 1.0, selection)
    callback             = ProtoSyn.Common.default_energy_step_callback(10)
    thermostat           = ProtoSyn.Drivers.get_linear_quench(1.0, 500)
    energy_function      = ProtoSyn.Common.default_energy_function()
    energy_function.clean_cache_every = 10
    
    monte_carlo          = ProtoSyn.Drivers.MonteCarlo(energy_function, dihedral_mutator, callback, 500, thermostat)

    function start_simulation(job_cards::RemoteChannel, results::RemoteChannel)
        while true
            println("Starting simulation on worker $(myid())")
            job  = take!(job_cards)
            pose = ProtoSyn.Peptides.load("data/2a3d.pdb")
            monte_carlo(pose)
            put!(results, pose)
        end
    end
end

Part of the set-up of a distributed code in the `start_simulation` function. This start and infine loop that continuously takes "job cards" from a RemoteChannel, performs the job, and returns the result to a results RemoteChannel. The job card can, optionally, contain instructions of extra information. In this case, it's just a measure of how many jobs were requested and are left in the queue. whenever a worker finishes a simulation, if extra jobs are queued up, it starts a new simulation. In order for this strategy to work, we first need to create and populate the jobs queue, as well as create the results RemoteChannel.

In [ ]:
N = 8
job_queue = RemoteChannel(() -> Channel{Int16}(N))
for job_n in 1:N
    put!(job_queue, Int16(job_n))
end
job_queue

In [ ]:
results_queue = RemoteChannel(() -> Channel{Any}(N))

We can now begin spawning the 8 simulations in our 4 workers, by calling the `start_simulation` method in each one of them.

In [ ]:
for p in workers()
    remote_do(start_simulation, p, job_queue, results_queue)
end

A second infinite loop continously checks the results RemoteChannel in order to retrieve the finished simulation results.

In [ ]:
begin
    local n = N
    results = []
    while n > 0
        result = take!(results_queue)
        push!(results, result)
        n = n - 1
    end
end